In [1]:
import pandas as pd
import torchtext
from torchtext.vocab import Vectors
from torchtext.legacy import data
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
from torch.autograd import Variable
from sklearn import metrics
import sys
sys.path.append("D:/Experiment")
from tqdm import tqdm
from MyKu import training
from MyKu import processing
from torchtext.vocab import Vectors
from spacy.lang.en import English

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [2]:
def global_max_pooling(tensor, dim, topk):
    """Global max pooling"""
    ret, _ = torch.topk(tensor, topk, dim)
    return ret

class textCNN_LSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hidden, num_layers, dropout, **kwargs):
        super(textCNN_LSTM, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        self.decoder = nn.Linear(num_hidden * 2, 2)
        self.conv2 = nn.Conv1d(embed_size, 100, 2, padding=1)
        self.conv3 = nn.Conv1d(embed_size, 100, 3, padding=2)
        self.maxpool1D = nn.MaxPool1d(4, stride=4)
        self.seq_model = nn.LSTM(100, num_hidden, bidirectional=True, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        inputs = inputs.permute(1, 0)
        embeddings = self.embedding(inputs)
        embeddings = embeddings.permute(0, 2, 1)
        new_conv2 = self.maxpool1D(self.conv2(embeddings))
        new_conv3 = self.maxpool1D(self.conv3(embeddings))
        encoding = self.maxpool1D(torch.cat([new_conv2, new_conv3], dim=2))
        # print('encoding.shape: ', encoding.shape)
        h_seq, _ = self.seq_model(encoding.permute(0, 2, 1))
        # print('h_seq.shape: ', h_seq.shape)
        global_h_seq=torch.squeeze(global_max_pooling(h_seq, 1, 1)) 
        # print('h_seq.shape: ', global_h_seq.shape)
        global_h_seq = self.dropout(global_h_seq)
        outputs = self.decoder(global_h_seq)
        return outputs

In [3]:
spacy_en = English()


def tokenizer(text):  # create a tokenizer function
    """
    定义分词操作
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]


def DataLoader():
    def tokenize(x): return x.split()

    TEXT = data.Field(sequential=True, tokenize=tokenizer,
                      lower=True, include_lengths=True, fix_length=40)
    LABEL = data.Field(sequential=False, use_vocab=False)
    # 假如train.csv文件并不是只有两列，比如1、3列是review和polarity，2列是我们不需要的数据，
    # 那么就要添加一个全是None的元组， fields列表存储的Field的顺序必须和csv文件中每一列的顺序对应，

    train_fields = [(None, None), ('tweet', TEXT), ('subtask_a', LABEL)]
    # train_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    train_data = data.TabularDataset(
        path='D:/Experiment/datasets/OLID/train.tsv',
        # path='D:/Experiment/datasets/EXIST2021/train.tsv',
        format='tsv',
        fields=train_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    test_fields = [(None, None), ('tweet', TEXT), ('label', LABEL)]
    # test_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    test_data = data.TabularDataset(
        path='D:/Experiment/datasets/OLID/testA.tsv',
        # path='D:/Experiment/datasets/EXIST2021/test.tsv',
        format='tsv',
        fields=test_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    return train_data, test_data, TEXT, LABEL

train_data, test_data, TEXT, LABEL = DataLoader()

vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                 max_size=400000,  # 单词表容量
                 vectors=vectors,  # 还有'glove.840B.300d'已经很多可以选
                 unk_init=torch.Tensor.normal_  # 初始化train_data中不存在预训练词向量词表中的单词
                 )


In [4]:

embed_size, num_hidden, kernel_sizes, nums_channels, num_layers, dropout = 300, 100, [3, 4, 5], [100, 100, 100], 1, 0.7
model = textCNN_LSTM(len(TEXT.vocab), embed_size, num_hidden, num_layers,dropout)
model.to(DEVICE)

def init_weights(m):
    if type(m) in (nn.Linear, nn.Conv1d):
        nn.init.xavier_uniform_(m.weight)

model.apply(init_weights);

pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
model.embedding.weight.data[PAD_IDX] = torch.zeros(300)

d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.7 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:
train_iter, test_iter = data.BucketIterator.splits(
    (train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x: len(x.tweet), device=DEVICE
)

In [6]:
def train(model, train_iter, optimizer, loss, epoch):
    model.train()
    epoch_loss = 0
    num_sample = 0
    correct = 0
    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", colour='red'):
        optimizer.zero_grad()
        text, text_len = batch.tweet
        label = batch.subtask_a
        # text, text_len = batch.text
        # label = batch.task1
        output = model(text)
        pred_y = torch.argmax(output, dim=1)
        correct += torch.sum(pred_y == label)
        l = loss(output, label)
        l.backward()
        epoch_loss += l.item()
        num_sample += len(batch)
        optimizer.step()
    print(
        f'\tTrain Loss: {epoch_loss / num_sample:.3f} | Train Acc: {correct.float() / num_sample* 100:.2f}%')


def test(model, test_iter):
    true_y, pred_y = [], []
    for batch in tqdm(test_iter, desc=f"Testing", colour='green'):
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        with torch.no_grad():
            output = model(text)
            pred_y.extend(output.argmax(dim=1).tolist())
            true_y.extend(label.tolist())
    print(metrics.confusion_matrix(true_y, pred_y))
    print(metrics.classification_report(true_y, pred_y))
    print(f'Acc : {metrics.accuracy_score(true_y, pred_y)}\t F1: {metrics.f1_score(true_y, pred_y, average="macro")}')

In [7]:
lr, num_epochs = 0.0001, 30
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
for epoch in range(1, num_epochs+1):
    train(model, train_iter, optimizer, loss, epoch)
    test(model, test_iter)

Training Epoch 1: 100%|██████████| 207/207 [00:02<00:00, 86.73it/s] 


	Train Loss: 0.010 | Train Acc: 66.35%


Testing: 100%|██████████| 14/14 [00:00<00:00, 449.76it/s]


[[618   2]
 [238   2]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       620
           1       0.50      0.01      0.02       240

    accuracy                           0.72       860
   macro avg       0.61      0.50      0.43       860
weighted avg       0.66      0.72      0.61       860

Acc : 0.7209302325581395	 F1: 0.4268959083033454


Training Epoch 2: 100%|██████████| 207/207 [00:01<00:00, 108.56it/s]


...

[[559  61]
 [116 124]]
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       620
           1       0.67      0.52      0.58       240

    accuracy                           0.79       860
   macro avg       0.75      0.71      0.72       860
weighted avg       0.78      0.79      0.79       860

Acc : 0.7941860465116279	 F1: 0.7234249375425845


Training Epoch 30: 100%|██████████| 207/207 [00:01<00:00, 113.35it/s]


	Train Loss: 0.000 | Train Acc: 99.65%


Testing: 100%|██████████| 14/14 [00:00<00:00, 479.08it/s]

[[561  59]
 [118 122]]
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       620
           1       0.67      0.51      0.58       240

    accuracy                           0.79       860
   macro avg       0.75      0.71      0.72       860
weighted avg       0.78      0.79      0.78       860

Acc : 0.7941860465116279	 F1: 0.7216568930238682
